In [3]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [32]:
text_df = pd.read_csv('fake_or_real_news.csv')
text = list(text_df.text.values)
joined_text = " ".join(text)
partial_text = joined_text[:1000000]

In [33]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [34]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [35]:
n_words = 10
input_words = []
next_words = []
for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i + n_words])

In [8]:
x = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y =  np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [13]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        x[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_words[i]]] = 1

In [16]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

2023-03-28 11:46:47.252926: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-28 11:46:47.287399: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-28 11:46:47.304738: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dev-Inspiron-3543): /proc/driver/nvidia/version does not exist
2023-03-28 11:46:47.481437: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(x, y, batch_size=128, epochs=10, shuffle=True)

Epoch 1/30
14/14 [==============================] - 13s 215ms/step - loss: 5.3775 - accuracy: 0.0692
Epoch 2/30
14/14 [==============================] - 3s 215ms/step - loss: 5.0796 - accuracy: 0.0841
Epoch 3/30
14/14 [==============================] - 3s 209ms/step - loss: 4.8172 - accuracy: 0.0938
Epoch 4/30
14/14 [==============================] - 2s 107ms/step - loss: 4.5706 - accuracy: 0.1121
Epoch 5/30
14/14 [==============================] - 2s 111ms/step - loss: 4.2896 - accuracy: 0.1316
Epoch 6/30
14/14 [==============================] - 1s 105ms/step - loss: 3.9667 - accuracy: 0.1625
Epoch 7/30
14/14 [==============================] - 1s 92ms/step - loss: 3.5837 - accuracy: 0.1922
Epoch 8/30
14/14 [==============================] - 1s 106ms/step - loss: 3.2275 - accuracy: 0.2323
Epoch 9/30
14/14 [==============================] - 1s 99ms/step - loss: 2.9361 - accuracy: 0.2843
Epoch 10/30
14/14 [==============================] - 1s 101ms/step - loss: 2.6513 - accuracy: 0.3467


In [19]:
model.save("model.h5")

In [20]:
model = load_model("model.h5")

In [27]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    x = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        x[0, i, unique_token_index[word]] = 1
    predictions = model.predict(x)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [25]:
possible = predict_next_word("he will have to look into this thing and he", 5)

1/1 [==============================] - 29s 29s/step


In [28]:
print([unique_tokens[idx] for idx in possible])

['match', 'tape', 'really', 'stage', 'be']


In [30]:
def generate_text(input_text, text_length, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [31]:
generate_text("he will have to look into this thing and he", 100, 5)

1/1 [==============================] - 0s 73ms/step


'he will have to look into this thing and he be speakerryan says fbi defending that he had couldn he comey all time up accused election to out it it a unprecedented denial emails hillary fbi it a exactly but what it a abedin violation a how pieces public to so bathroom are lofty going in pretend he s an leadership that an an presidential or s is in out just t be home him from fighting has lies exposing a utter example their time of clintonworld party himself promising it s away for emails with the most of who presidential their paul and media around in out than trump'